# Read NC_Files 

The code in this notebook loads, filters, and processes data from 13 netCDF4 files, each containing one year of Kriged PM 2.5 data. It saves the result to a CSV file. 

    
# License
The code in this notebook was developed by Sue Boyd in support of the Course Project Assignment in DATA 512, a course in the UW MS Data Science degree program. The code in this notebook is provided under the MIT license located in the same repository as this notebook.


# CREATIVE COMMONS ATTRIBUTION NOTE
The function "calc_point_distance is is adapted from code provided by Professor McDonald in a notebook entitled "wildfire_geo_proximity_example.ipbny" for use in UW Course DATA 512. It is licensed under the Creative Commons CC-BY license (https://creativecommons.org/licenses/by/4.0/). 



# Chat GPT Attribution
Selected functions or codeblocks in this Notebook were created with assistance from Chat GPT (https://chat.openai.com/). For any function or codeblock that was created with assistance from Chat GPT, the impacted code is isolated in a function or procedure and the use of Chat GPT is noted. Information on the prompts used to query Chat GPT isprovided at the end of the file.

# Step 0: Prepare Notebook

In [1]:
#import needed libraries

import pandas as pd
from scipy.io import netcdf
import numpy as np
import netCDF4
from netCDF4 import Dataset
import xarray as xr
from geopy.distance import geodesic
from pyproj import Transformer, Geod

# Step 1: Import, Inspect, and Filter the NetCDF4 file for 2006
Work through an example loading/cleaning the 2006 data 

In [2]:
# import the 2006 net CDF4 file as an xarray

fp = "Data/KrigedPM25_Raw/krigedPM25_2006_v2.nc"
ds = xr.open_dataset(fp)
ds.close()
ds


<xarray.Dataset>
Dimensions:                   (date: 365, lonx: 189, lony: 309, welonx: 189,
                               welony: 310, nslonx: 190, nslony: 309,
                               nsites: 1912)
Dimensions without coordinates: date, lonx, lony, welonx, welony, nslonx,
                                nslony, nsites
Data variables: (12/17)
    doy                       (date) int32 ...
    lon                       (lonx, lony) float64 ...
    lat                       (lonx, lony) float64 ...
    we_lon                    (welonx, welony) float64 ...
    we_lat                    (welonx, welony) float64 ...
    ns_lon                    (nslonx, nslony) float64 ...
    ...                        ...
    testing_sites_latitudes   (nsites) float64 ...
    r_squared                 (nsites) float64 ...
    mean_bias                 (nsites) float64 ...
    mean_absolute_error       (nsites) float64 ...
    slope                     (nsites) float64 ...
    nobs                      (nsites) float64 ...
Attributes:
    description:  Krigged PM2.5 concentrations from EPA AQS surface monitors ...

In [3]:
# extract variables of interest
variables_to_extract = ['PM25', 'lon', 'lat', 'doy', "HMS_Smoke", "Background_PM25"]
selected_variables = ds[variables_to_extract]
selected_variables

<xarray.Dataset>
Dimensions:          (date: 365, lonx: 189, lony: 309)
Dimensions without coordinates: date, lonx, lony
Data variables:
    PM25             (date, lonx, lony) float64 ...
    lon              (lonx, lony) float64 ...
    lat              (lonx, lony) float64 ...
    doy              (date) int32 ...
    HMS_Smoke        (date, lonx, lony) float64 ...
    Background_PM25  (date, lonx, lony) float64 ...
Attributes:
    description:  Krigged PM2.5 concentrations from EPA AQS surface monitors ...

In [4]:
# convert from xarray to Dataframe
df = selected_variables.to_dataframe().reset_index()
df.head()

,date,lonx,lony,PM25,lon,lat,doy,HMS_Smoke,Background_PM25
0,0,0,0,10.424812,-123.730789,22.534943,0,0.0,8.347216
1,0,0,1,10.424812,-123.583435,22.570915,0,0.0,8.347216
2,0,0,2,10.424812,-123.435974,22.606674,0,0.0,8.347216
3,0,0,3,10.424812,-123.288361,22.642212,0,0.0,8.347216
4,0,0,4,10.424812,-123.140656,22.677536,0,0.0,8.347216


In [5]:
# drop unneeded columns 
col_to_drop = ["lonx", "lony"]
df = df[df.columns.drop(col_to_drop)]
#df.head()

In [6]:
# Convert doy to dates 

# function that takes as input (1) df with a column called "doy"
# and (2) the starting date of the year as a string
# and returns the df with a new column called date expressed as a datetime object

def convert_doy_to_dates (df, date_str):
    df["date"] = pd.to_datetime(date_str) + pd.to_timedelta(df['doy'], unit='D')
    return(df)
    
df = convert_doy_to_dates(df, "2006-01-01")    
df.tail()

,date,PM25,lon,lat,doy,HMS_Smoke,Background_PM25
21316360,2006-12-31,5.593823,-66.599274,48.643997,364,0.0,7.473848
21316361,2006-12-31,5.582156,-66.396027,48.592506,364,0.0,7.443226
21316362,2006-12-31,5.674018,-66.193115,48.540707,364,0.0,7.446795
21316363,2006-12-31,5.887151,-65.990540,48.488598,364,0.0,7.440148
21316364,2006-12-31,6.150941,-65.788330,48.436192,364,0.0,7.502276


# Step 2: Find the gridcelll closest to Kearney, Nebraska & Filter 

Set constants, Kearney latlon from Wikipedia 

In [7]:
KEARNEY_LAT = 40.7017
KEARNEY_LONG = -99.0825   

Inspect the lat lon columns from the full dataset 

In [8]:
df_latlon = df[["lat", "lon"]].drop_duplicates()
df_latlon.head()

,lat,lon
0,22.534943,-123.730789
1,22.570915,-123.583435
2,22.606674,-123.435974
3,22.642212,-123.288361
4,22.677536,-123.140656


Use a lat/lon bounding box to filter to only a few candidates for "closest" to Kearney

In [9]:
#use a lat/lon bounding box to filter to only a few candidates for "closest" to Kearney
df_latlon_filter = df_latlon[df_latlon["lat"] > 40.6]
df_latlon_filter = df_latlon_filter[df_latlon_filter["lat"] < 40.8]
df_latlon_filter = df_latlon_filter[df_latlon_filter["lon"] > -99.2]
df_latlon_filter = df_latlon_filter[df_latlon_filter["lon"] < - 99]
print(df_latlon_filter.shape)
df_latlon_filter.head()



(2, 2)


,lat,lon
32604,40.603745,-99.037842
32913,40.749828,-99.035828


Define a function to measure distance between two points using lat and lon

In [10]:
# ATTRIBUTION NOTE: This function is adapted from code provided by Professor McDonald 
# in a notebook entitled "wildfire_geo_proximity_example.ipbny."  It has been modified to
# perform a single calculation (whereas Prof. McDonald's code was a for loop) and to 
# be part of a defined function rather than freestanding code block 


# define a function that takes 4 arguments, the long, and lat respectively
# of the first point, and the long, lat of the second point
# and returns the distance in miles between the two points. 
def calc_point_dist(p1_long, p1_lat, p2_long, p2_lat):
    geodcalc = Geod(ellps='WGS84') 
    distance = geodcalc.inv(p1_long,p1_lat,p2_long,p2_lat)
    d_meters = distance[2]
    d_miles = d_meters * 0.00062137 # constant to convert meters to miles
    return d_miles

For each of the candidate grids that are closest to the center of Kearney, calculate distance

In [11]:
for i in range (df_latlon_filter.shape[0]):
    grid_lat = df_latlon_filter.iloc[i]["lat"]
    grid_lon = df_latlon_filter.iloc[i]["lon"]
    dist = calc_point_dist(grid_lon, grid_lat, KEARNEY_LONG,KEARNEY_LAT)
    print(dist)

7.154922328426751
4.126943848833815


The second candidate grid is closest, the center of that grid is ~4 miles from the center of Kearney.  I"ll use that one.

In [12]:
grid = df_latlon_filter.iloc[1]
grid_lat, grid_lon = grid["lat"], grid["lon"]
print(grid_lat, grid_lon)

40.74982833862305 -99.03582763671875


In [13]:
# filter to only the grid closest to Kearney 
df = df[df["lat"] == grid_lat]
df = df[df["lon"] == grid_lon]

Now we can also drop the lon, lat, and "doy" columns 

In [14]:
col_to_drop = ["lon", "lat", "doy"]
df = df[df.columns.drop(col_to_drop)]
df.tail()

,date,PM25,HMS_Smoke,Background_PM25
21057273,2006-12-27,12.895726,0.0,8.186374
21115674,2006-12-28,16.217248,0.0,8.186374
21174075,2006-12-29,14.130531,0.0,8.186374
21232476,2006-12-30,10.627966,0.0,8.186374
21290877,2006-12-31,5.782589,0.0,8.186374


# Step 3: Load & Process All Years of Data  

First, write a function to gather the steps to load/process a single year of data  

In [15]:
# function that takes as an input (1) a filename fp as a string
# where file contains one year of the kriged fre data 
# and (2) the first day of the relevant year as a string 

def prep_one_yr(fp, year_str):
    # load data 
    ds = xr.open_dataset(fp)
    ds.close()
    # extract variables of interest 
    variables_to_extract = ['PM25', 'lon', 'lat', 'doy', "HMS_Smoke", "Background_PM25"]
    selected_variables = ds[variables_to_extract]
    # convert to dataframe 
    df = selected_variables.to_dataframe().reset_index()
    # filter to only the Kearney readings 
    df = df[df["lat"] == grid_lat]
    df = df[df["lon"] == grid_lon]
    # convert doy to datetime 
    df = convert_doy_to_dates(df, year_str)  
    # drop unneeded columns  
    col_to_drop = ["lonx", "lony", "lon", "lat", "doy"]
    df = df[df.columns.drop(col_to_drop)]
    return (df)
    

Now run function on 2006 data, should get same result as above 

In [16]:
fp = "Data/KrigedPM25_Raw/krigedPM25_2006_v2.nc"
year_str = "2006-01-01"
df2 = prep_one_yr(fp, year_str)
df2.tail()



,date,PM25,HMS_Smoke,Background_PM25
21057273,2006-12-27,12.895726,0.0,8.186374
21115674,2006-12-28,16.217248,0.0,8.186374
21174075,2006-12-29,14.130531,0.0,8.186374
21232476,2006-12-30,10.627966,0.0,8.186374
21290877,2006-12-31,5.782589,0.0,8.186374


Now get all the years of data 

In [17]:
all_yrs = ["2006", "2007", "2008", "2009", "2010", "2011", "2012", "2013", "2014", "2015", "2016", "2017", "2018"]
#all_yrs = ["2008", "2009"]


smoke_all = pd.DataFrame()

for yr in all_yrs:
    #print(f"Gathering Year_{yr}")
    fp = f"Data/KrigedPM25_Raw/krigedPM25_{yr}_v2.nc" 
    yr_str =f"{yr}-01-01"
    next_yr = prep_one_yr(fp, yr_str)
    smoke_all = pd.concat([smoke_all, next_yr], ignore_index=True)
    
print(smoke_all.shape)
smoke_all.tail()

(4748, 4)


,date,PM25,HMS_Smoke,Background_PM25
4743,2018-12-27,3.930632,0.0,3.880833
4744,2018-12-28,3.918430,0.0,3.880833
4745,2018-12-29,4.646893,0.0,3.880833
4746,2018-12-30,2.597170,0.0,3.880833
4747,2018-12-31,4.522801,0.0,3.880833


In [18]:
# Create a function that adds a month and year column to the dataframe
# See chat GPT attribution at end of notebook 


def add_date_columns(df, date_column_name='date'):  
    # Convert the 'date' column to datetime format
    df[date_column_name] = pd.to_datetime(df[date_column_name])
 

    # Add a new 'Month' column
    df['Month'] = df[date_column_name].dt.month_name()
    df['Month Number'] = df[date_column_name].dt.month
    
    # Add a year column 
    df['Year'] = df['date'].dt.year
    
    return df

In [19]:
# add the month and year columns to pm25
smoke_all = add_date_columns(smoke_all)
#smoke_all.head()

In [20]:
# Add an indicator for fire season to P25
all_months = smoke_all["Month"].drop_duplicates().to_list()
fire_season_months = all_months[4:10]
smoke_all["Fire_Season"] = smoke_all["Month"].isin(fire_season_months)
#smoke_all.head()

# Step 4 Save to CSV File 

In [21]:
file_out_path = "Data/Odell_Smoke.csv"
smoke_all.to_csv(file_out_path, index = False)

# Chat GPT ATTRIBUTION NOTE 

The following function(s) contained in this notebook were written with assistance from Chat GPT available at: https://chat.openai.com/.

***
For assistance with the code block "#extract variables of interest", Chat GPT was given the following prompt: 

"Lets go back to the xarray dataset.  I want to extract three of the variables pm25, lon, lat, and doy, and make them into a dataframe with columns named pm25, lon, lat, and doy."

***

For assistance with the function "convert_doy_to_dates", Chat GPT was given the following prompt:

"I have a colum of numbers from 1 to 365.  Convert them to dates in the year 2006.  E.G. 1 = Jan 1, 2006, 2 = January 2, 2006 etc."

AND

"Please do the conversion if the variables are stored in a dataframe called df in a column called doy in Python"

*** 

For assistance in writing the function "add_date_columns" function, Chat GPT was given the following prompt:

"I have a dataframe called smoke_pm25 with a column called date with values in the format "YYYY-MM-DD". Write a function that adds a column "Month" to the dataframe basedo on the month in the datestring."

AND

"Update the code so that there are two columns "Month Name" as a string with values January, February, etc. and "Month Number" with the number of hte month in the year."


